In [ ]:
# Install in python < 3.9 environment

In [1]:
!pip3 install fair-esm
!pip install "fair-esm[esmfold]"
!pip install 'dllogger @ git+https://github.com/NVIDIA/dllogger.git'
!pip install 'openfold @ git+https://github.com/aqlaboratory/openfold.git@4b41059694619831a7db195b7e0988fc4ff3a307'
!pip install biotite
!pip install pytorch-lightning==1.8.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.3/510.3 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 40.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 60.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torch to determine which version is compatible with other requirements. This cou

In [2]:
import os
import tempfile
import numpy as np
import torch
from Bio import SeqIO
import esm
import biotite.structure.io as bsio
import biotite.structure as struc

In [3]:
data_dir = "/workspace/plm-train/data/netsolp/PSI_Biology"

In [4]:
model = esm.pretrained.esmfold_v1()
model = model.eval().cuda()

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esmfold_3B_v1.pt" to /root/.cache/torch/hub/checkpoints/esmfold_3B_v1.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t36_3B_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t36_3B_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t36_3B_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t36_3B_UR50D-contact-regression.pt


# Training Data: pET_full_without_his_tag 

In [48]:
# Read fasta file and iterate through all sequences
fasta_file = os.path.join(data_dir, "pET_full_without_his_tag.fa")
fasta_sequences = SeqIO.parse(open(fasta_file), "fasta")


# Temporary directory to store the pdb files
protein_names = []
protein_sasa = []
chunk_counts = 0
with tempfile.TemporaryDirectory() as tmpdirname:
    print('Created temporary directory:', tmpdirname)
    # Iterate through all the fasta sequences
    for fasta in fasta_sequences:
        sel_name, sel_seq = fasta.id, str(fasta.seq)
        with torch.no_grad():
            output = model.infer_pdb(sel_seq)
        tmp_pdb_file = os.path.join(tmpdirname, "result.pdb")
        with open(tmp_pdb_file, "w") as f:
            f.write(output)

        atom_array = bsio.load_structure(tmp_pdb_file)
        atom_array = atom_array[atom_array.hetero == False]
        assert len(np.unique(atom_array.res_id)) == len(sel_seq)
        assert np.array_equal(
            np.sort(np.unique(atom_array.res_id)), np.arange(1, atom_array.res_id.max() + 1)
        )
        atom_sasa = struc.sasa(atom_array)
        # Sum up SASA for each residue in chain
        # res_sasa_chain = struc.apply_residue_wise(sel_chain, atom_sasa_chain, np.sum)
        res_sasa = np.bincount(atom_array.res_id,  weights=atom_sasa)[1:]  # From ESM3
        assert res_sasa.shape[0] == len(sel_seq)
        
        # Add to list
        protein_names.append(sel_name)
        protein_sasa.append(res_sasa)

        # Save periodic results after every 500 proteins
        chunk_size = 100
        if len(protein_names) % chunk_size == 0:
            suffix_name_1 = str(chunk_counts + int(np.floor(len(protein_names)/chunk_size) - 1))
            suffix_name_2 = str(chunk_counts + int(np.floor(len(protein_names)/chunk_size)))
            npz_file_name = os.path.join(data_dir, "sasa_data", f"pET_full_without_his_tag_sasa_data_{suffix_name_1}_{suffix_name_2}.npz")
            np.savez(npz_file_name, **dict(zip(protein_names, protein_sasa)))

            # Reset the lists
            protein_names = []
            protein_sasa = []
            chunk_counts+=1



Created temporary directory: /tmp/tmppfrzm8am


# Consolidate all the chunks

In [15]:
# Read through all the npz chunks and consolidate into one npz file
npz_files = os.listdir(os.path.join(data_dir, "sasa_data"))
print(len(npz_files))
protein_names = []
protein_sasa = []
for npz_file in npz_files:
    data = np.load(os.path.join(data_dir, "sasa_data", npz_file))
    protein_names.extend(list(data.keys()))
    protein_sasa.extend(list(data.values()))

# save the consolidated data
np.savez(os.path.join(data_dir, "pET_full_without_his_tag_sasa_data_consolidated.npz"), **dict(zip(protein_names, protein_sasa)))



122


In [3]:
# Load the consolidated data
data = np.load(os.path.join(data_dir, "pET_full_without_his_tag_sasa_data_consolidated.npz"))
protein_names = list(data.keys())
protein_sasa = list(data.values())
print(len(protein_names))

12200


In [11]:
# Read fasta file and iterate through all sequences
fasta_file = os.path.join(data_dir, "pET_full_without_his_tag.fa")
fasta_sequences = SeqIO.parse(open(fasta_file), "fasta")

# Convert fasta_sequences to dictionary
fasta_dict = {}
for fasta in fasta_sequences:
    fasta_dict[fasta.id] = str(fasta.seq)

# Create a new fasta file with the fasta ids of protein_names
new_fasta_file = os.path.join(data_dir, "pET_full_without_his_tag_consolidated.fasta")
with open(new_fasta_file, "w") as f:
    for protein_name in protein_names:
        f.write(f">{protein_name}\n{fasta_dict[protein_name]}\n")



# Test Data: NESG_Price Data

In [13]:
# Read fasta file and iterate through all sequences
fasta_file = os.path.join(data_dir, "NESG_testset.fasta")
fasta_sequences = SeqIO.parse(open(fasta_file), "fasta")


# Temporary directory to store the pdb files
protein_names = []
protein_sasa = []
failed_sequences = []
chunk_counts = 0
with tempfile.TemporaryDirectory() as tmpdirname:
    print('Created temporary directory:', tmpdirname)
    # Iterate through all the fasta sequences
    for fasta in fasta_sequences:
        sel_name, sel_seq = fasta.id, str(fasta.seq)
        with torch.no_grad():
            output = model.infer_pdb(sel_seq)
        tmp_pdb_file = os.path.join(tmpdirname, "result.pdb")
        with open(tmp_pdb_file, "w") as f:
            f.write(output)

        atom_array = bsio.load_structure(tmp_pdb_file)
        atom_array = atom_array[atom_array.hetero == False]
        if len(np.unique(atom_array.res_id)) != len(sel_seq):
            # Go to next fasta
            failed_sequences.append(sel_seq)
            continue
        assert len(np.unique(atom_array.res_id)) == len(sel_seq)
        assert np.array_equal(
            np.sort(np.unique(atom_array.res_id)), np.arange(1, atom_array.res_id.max() + 1)
        )
        atom_sasa = struc.sasa(atom_array)
        # Sum up SASA for each residue in chain
        # res_sasa_chain = struc.apply_residue_wise(sel_chain, atom_sasa_chain, np.sum)
        res_sasa = np.bincount(atom_array.res_id,  weights=atom_sasa)[1:]  # From ESM3
        assert res_sasa.shape[0] == len(sel_seq)
        
        # Add to list
        protein_names.append(sel_name)
        protein_sasa.append(res_sasa)

        # Save periodic results after every 500 proteins
        chunk_size = 100
        if len(protein_names) % chunk_size == 0:
            suffix_name_1 = str(chunk_counts + int(np.floor(len(protein_names)/chunk_size) - 1))
            suffix_name_2 = str(chunk_counts + int(np.floor(len(protein_names)/chunk_size)))
            npz_file_name = os.path.join(data_dir, "sasa_data_NESG_testset", f"NESG_testset_sasa_data_{suffix_name_1}_{suffix_name_2}.npz")
            np.savez(npz_file_name, **dict(zip(protein_names, protein_sasa)))

            # Reset the lists
            protein_names = []
            protein_sasa = []
            chunk_counts+=1

Created temporary directory: /tmp/tmpbfyvk7xu


In [15]:
len(failed_sequences)

1

In [19]:
# Read through all the npz chunks and consolidate into one npz file
npz_files = os.listdir(os.path.join(data_dir, "sasa_data_NESG_testset"))
print(len(npz_files))
protein_names = []
protein_sasa = []
for npz_file in npz_files:
    data = np.load(os.path.join(data_dir, "sasa_data_NESG_testset", npz_file))
    protein_names.extend(list(data.keys()))
    protein_sasa.extend(list(data.values()))

# save the consolidated data
np.savez(os.path.join(data_dir, "NESG_testset_sasa_data_consolidated.npz"), **dict(zip(protein_names, protein_sasa)))



13


In [20]:
# Load the consolidated data
data = np.load(os.path.join(data_dir, "NESG_testset_sasa_data_consolidated.npz"))
protein_names = list(data.keys())
protein_sasa = list(data.values())
print(len(protein_names))

1300


In [22]:
# Read fasta file and iterate through all sequences
fasta_file = os.path.join(data_dir, "NESG_testset.fasta")
fasta_sequences = SeqIO.parse(open(fasta_file), "fasta")

# Convert fasta_sequences to dictionary
fasta_dict = {}
for fasta in fasta_sequences:
    fasta_dict[fasta.id] = str(fasta.seq)

# Create a new fasta file with the fasta ids of protein_names
new_fasta_file = os.path.join(data_dir, "NESG_testset_sasa_data_consolidated.fasta")
with open(new_fasta_file, "w") as f:
    for protein_name in protein_names:
        f.write(f">{protein_name}\n{fasta_dict[protein_name]}\n")

